## **Context**

This notebook aims to develop a data-driven **forecasting framework** based on three years of **Chicago 311 Service Requests** records, providing a practical reference for **operational planning** in similar real-world applications.

The analysis evalutes three different forecasting approaches:

- **ARIMA/SARIMA** — statistical time-series models, designed to capture trends and seasonality
- **Prophet** — decomposable model that explicitly models trens, seasonality and calendar effects.
- **LightGBM** — gradient boosting framework applied to temporal feature engineering (lags, rolling statistics, calendar variables) to capture nonlinear patterns and external influences

### **311 Service Requests**

The City of Chicago operates a centralized **311 Service Requests system**, allowing residents to report non-emergency issues related to public infrastructure and city services. 

Each request represents operational demand that must be addressed by city departments using **finite crews, equipment, and budget**.

### **Capacity x Availability Over Time**

One of the **primary operational target** of such service provider is the hability to deliver:
- **the right service**;
- **to the right requester**;
- **with the right capacity**;
- **within the SLA time target**.

These principles have** wide application across most industries** and have **proven to be reliable**, provided they are a**pplied correctly and supported by adequate data**. 

In real life, service request volumes can vary significantly over time, driven by seasonality and differences across service types. 

Despite this variability, staffing and resource allocation decisions are commonly made in a **manual and reactive manner**, relying on historical averages and short-term intuition rather than systematic forecasts, often leading to operational inefficiency such as:
- **Understaffing** during **peak periods**, resulting in:
    - increasing service backlogs;
    - delayed response times  
- **Overstaffing** during **low-demand periods**, leading to:
    - **idle** crews;
    - **inefficient** resources use  

### **Stakeholders**

The outputs of this analysis are **intended to support** decisions made by:

- **Operations managers** responsible for crew scheduling  
- **Department leads** coordinating maintenance activities  
- **Budget and planning teams** overseeing short-term operational efficiency

## **Data Understanding & Exploratory Analysis**

A **comprehensive data understanding and exploratory analysis** were conducted in a [previous notebook](); the focus here is to further **examine and prepare the dataset specifically for the forecasting application** described above.

## **Data Overview**

This section examines the **raw demand signal** to characterize its **temporal dynamics, seasonality, and variation** across service types.

As shown in the previous notebook, the dataset contains over 7 million records and more than 30 columns. Therefore, to ensure faster and more stable analysis, DuckDB queries are preferred over repeatedly loading the full dataset into multiple pandas objects

In [16]:
# Importing Python libraries
import numpy as np
from pathlib import Path
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geoviews as gv
import cartopy.crs as ccrs
from holoviews.operation.datashader import datashade
from colorcet import fire
import plotly.graph_objects as go
from statsmodels.tsa.stattools import adfuller

In [3]:
# Initialize DuckDB database
con = duckdb.connect() 

In [4]:
# Overview 
df = con.execute("""
    SELECT 
            SR_NUMBER, 
            SR_TYPE, 
            CREATED_DATE, 
            LAST_MODIFIED_DATE, 
            CLOSED_DATE

    FROM 'data/processed/eda.parquet'
""").df()

df.info()

# Date time columns
# CREATED_DATE          -  datetime64[us]
# LAST_MODIFIED_DATE    -  datetime64[us]
# CLOSED_DATE           -  datetime64[us]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7645761 entries, 0 to 7645760
Data columns (total 5 columns):
 #   Column              Dtype         
---  ------              -----         
 0   SR_NUMBER           object        
 1   SR_TYPE             object        
 2   CREATED_DATE        datetime64[us]
 3   LAST_MODIFIED_DATE  datetime64[us]
 4   CLOSED_DATE         datetime64[us]
dtypes: datetime64[us](3), object(2)
memory usage: 291.7+ MB


In [5]:
# Weekly aggregation
df_weekly = (
    df
    .assign(week=lambda x: x["CREATED_DATE"].dt.to_period("W").dt.start_time)
    .groupby("week")
    .size()
    .rename("n_requests")
    .reset_index()
    .sort_values("week")
)

df_weekly.head(15)

,week,n_requests
0,2021-12-27,5770
1,2022-01-03,31579
2,2022-01-10,32860
3,2022-01-17,30880
4,2022-01-24,30513
5,2022-01-31,32194
6,2022-02-07,31828
7,2022-02-14,33133
8,2022-02-21,34863
9,2022-02-28,38504


In [6]:
# Ploting
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_weekly["week"],
        y=df_weekly["n_requests"],
        mode="lines",
        line=dict(width=1),
        name="Weekly Requests"
    )
)

fig.update_layout(
    title="Weekly Total 311 Service Requests",
    xaxis_title="Week",
    yaxis_title="Number of Requests",
    height=400,
    template="plotly_dark"
)

fig.show()

# Clear sazonal peaks and valleys, as mentioned and the previous EDA
# Steadly growth with annual maximuns and minimuns

In [7]:
# monthly aggregation (mm/aa)
df_monthly = (
    df
    .assign(month=lambda x: x["CREATED_DATE"].dt.strftime("%m/%y"))
    .groupby("month")
    .size()
    .rename("n_requests")
    .reset_index()
)

# ordenar cronologicamente corretamente
df_monthly = (
    df_monthly
    .assign(month_sort_key=lambda x: pd.to_datetime(x["month"], format="%m/%y"))
    .sort_values("month_sort_key")
    .drop(columns="month_sort_key")
)

In [8]:
# Ploting
months = df_monthly["month"].astype(str).tolist()
year_change_marks = [m for m in months if m.startswith("01/")][1:]

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_monthly["month"],
        y=df_monthly["n_requests"],
        mode="lines",
        line=dict(width=1),
        name="Monthly Requests"
    )
)


for m in year_change_marks:
    fig.add_vline(
        x=m,
        line_width=2,
        line_color="red",
        line_dash="solid",
        opacity=0.9
    )

fig.update_layout(
    title="Monthly Total 311 Service Requests",
    xaxis_title="Month (mm/yy)",
    yaxis_title="Number of Requests",
    height=400,
    template="plotly_dark"
)

fig.show()

# Peaks around summer
# Valleys around winter
# Distinct outlier in 05/25

In [9]:

# service heatmap over time
df_heat = (
    df
    .assign(month=lambda x: x["CREATED_DATE"].dt.to_period("M").dt.to_timestamp())
    .groupby(["SR_TYPE", "month"])
    .size()
    .rename("n")
    .reset_index()
)

In [10]:
# pivot
pivot = (
    df_heat
    .pivot(index="SR_TYPE", columns="month", values="n")
    .fillna(0)
)

# percentage
pivot_pct = pivot.div(pivot.sum(axis=1), axis=0)

# top frequency
top = pivot.sum(axis=1).sort_values(ascending=False).index[:40]
pivot_pct = pivot_pct.loc[top]

# seasonal strength 
season_strength = np.sort(pivot_pct.values, axis=1).std(axis=1)

# order: more seazonal to less seazonal
order = pivot_pct.index[np.argsort(-season_strength)]
pivot_pct = pivot_pct.loc[order]

In [11]:
fig = go.Figure(
    data=go.Heatmap(
        z=pivot_pct.values,
        x=pivot_pct.columns,
        y=pivot_pct.index,
        colorscale="Inferno",
        colorbar=dict(title="Percentage"),
        zmin=0,
        zmax=0.3   # ↓ diminua para aumentar contraste
    )
)

for i in range(12, len(pivot_pct.columns), 12):
    fig.add_vline(
        x=pivot_pct.columns[i],
        line_width=.5,
        line_color="red"
    )

fig.update_layout(
    title="Service Type Monthly Intensity (% within type)",
    yaxis_title="Service Type",
    height=800,
    width=800,
    template="plotly_dark"
)

fig.update_yaxes(autorange="reversed")

fig.show()

# Manly 3 naturaly observable behaviors:

# 1) "Non sazonal" related nature:
# ==========================
#   -> Likely accounts for most of the requests ("311 INFORMATION ONLY CALL" 
#      accounts for almost 40% of the SR_TYPES)
#   -> Also likely hidding further sazonal behavior, due to the general nature
#      implied in the type name
#   -> Finance Park: Code implemented in March 2024 describing illegal parking 

# 2) "Winter" related nature:
# ==========================
#   -> Types that happened during winter:
#       - "Ice and Snow"  
#       - "Check for Leak" and "Plumbing Violation"
#   -> Types that happened right after winter:
#       - "Water on the street"
#       - "Pothole in street" and "Pothole in alley" -> Maybe related to the thermal expansion of melting ice?

# 3) "Summer" related nature:
# ==========================
#   -> Types that happened during summer:
#       - "Weed" related
#       - "Tree" related
#       - "Rodent" and "Animal" related
#       - "Open Fire Hydrant" -> News from aug/2023 read :"Hundrends of Chicagoans opened fire hydrants for relief 
#                                from heat" and the volume have been increasing since 2022 in every july
#       - "Water on basement"
#       - "Sanitation" and "Garbage" related
#       - "Sidewalk"


### **ARIMA/SARIMA Sustentability Validation**

In [ ]:
# ARIMA (Autoregressive Integrated Moving Average)

# "Can we explain the present from the past?"

# Assumptions:
#   - The past contains useful information
#   - Relationships are relatively stable over time
#   - The system is not strongly chaotic (short-term dependence dominates)

# AR - AutoRegressive
#      Current values depend on past observed values.

# I  - Integrated
#      The series may be cumulative (e.g., trend-driven).
#      We difference: current value minus previous value
#      to remove accumulation and stabilize the series.

# MA - Moving Average
#      Current values are adjusted using past forecast errors.

# SARIMA (Seasonal Autoregressive Integrated Moving Average)

# "Is there a repeating pattern over time?"
# If its yearly, 52 weeks...



In [ ]:
series = df_weekly["n_requests"].astype(float)

# ----- Rolling stats -----
rolling_mean = series.rolling(12).mean()
rolling_std  = series.rolling(12).std()

# bandas ±2 std
upper_band = rolling_mean + 2 * rolling_std
lower_band = rolling_mean - 2 * rolling_std

fig1 = go.Figure()

# série original
fig1.add_trace(go.Scatter(
    x=df_weekly["week"],
    y=series,
    mode="lines",
    name="Original",
    opacity=0.4
))

# rolling mean
fig1.add_trace(go.Scatter(
    x=df_weekly["week"],
    y=rolling_mean,
    mode="lines",
    name="Rolling Mean (12w)"
))

# banda superior
fig1.add_trace(go.Scatter(
    x=df_weekly["week"],
    y=upper_band,
    mode="lines",
    line=dict(dash="dot",
             color="orange"),
    name="+2 Std",
    line_width= .5,

))

# banda inferior
fig1.add_trace(go.Scatter(
    x=df_weekly["week"],
    y=lower_band,
    mode="lines",
    line=dict(dash="dot",
              color="orange"),
    name="- 2 Std",
    line_width= .5
))

fig1.update_layout(
    title="Signal Stability (Rolling Mean ± 2 Std)",
    xaxis_title="Week",
    yaxis_title="Value",
    height=350,
    template="plotly_dark"
)

fig1.show()

# % ± 2 std 
valid_mask = (~rolling_mean.isna()) & (~rolling_std.isna())

inside_band = (
    (series >= lower_band) &
    (series <= upper_band) &
    valid_mask
)

pct_inside = inside_band.sum() / valid_mask.sum() * 100

pct_inside.round(2)

# ARIMA/SARIMA are linear models with memory, so they can be sensitive to large variance shifts.
# Even without assuming normality, having ~92% of points within the rolling ±2σ band suggests a
# relatively controlled volatility and a consistent pattern that these models can learn.


# REVIEW PLOT

np.float64(92.12)

In [ ]:
# The Autocorrelation Function (ACF) measures the linear relationship
# between the series and its past values at different lags.

# Where the inital question was:

# "Can we explain the present from the past?"

# ACF asks:
# "How similar is the present to the past?"

# Basic concepts:
# lag -> "time pattern unity" (e.g. for weeks, lag=1, last week, lag=52, same week last year)
# AutoCorrelation (ACF) -> Correlation between now and klag ago

# Practical (and oversimplified) analysis:
# signal -> positive = same direction, negative = inverted
# anything below .3 is "irrelevant"
# .3-.5 is "weak" 
# .5-.7 is "strong"
# over .7 is "very strong"
# none of these should be decided alone, the context within the time series is very important

#


In [50]:
import numpy as np
import plotly.graph_objects as go

lags = 104
acf_values = [series.autocorr(lag=i) for i in range(lags)]

fig2 = go.Figure()

# linhas ±0.5 (atrás)
fig2.add_hline(y=0.5,  line=dict(color="red", width=.6), layer="below")
fig2.add_hline(y=-0.5, line=dict(color="red", width=.6), layer="below")

# linha zero (atrás)
fig2.add_hline(y=0, line=dict(color="white",  width=1))

# barras
fig2.add_trace(go.Bar(
    x=list(range(lags)),
    y=acf_values,
    name="ACF"
))

fig2.update_layout(
    title="Autocorrelation Function",
    xaxis_title="Lag (weeks)",
    yaxis_title="Autocorrelation",
    height=350,
    template="plotly_dark"
)

# eixo Y fixo em [-1, 1]
fig2.update_yaxes(range=[-1, 1])

fig2.update_xaxes(showgrid=False)
fig2.update_yaxes(showgrid=False)

fig2.show()

# The ACF shows how strongly the series correlates with itself at past lags.
# A slow early decay implies persistent short-term memory (recent weeks matter).
# A clear rise around lag ~52 suggests yearly seasonality, supporting SARIMA.

In [ ]:
# The Augmented Dickey-Fuller (ADF) test evaluates whether the series is stationary.
#
# ARIMA models assume stationarity (constant mean and variance over time).
# If the series has a unit root (non-stationary), differencing is required.
#
# H0: The series has a unit root (non-stationary).
# H1: The series is stationary.

from statsmodels.tsa.stattools import adfuller
import plotly.graph_objects as go

series = df_weekly["n_requests"].astype(float)

# Plot the series (context for the test)
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_weekly["week"],
    y=series,
    mode="lines",
    name="Weekly Requests"
))

fig.update_layout(
    title="Weekly Demand (Context for Stationarity Test)",
    xaxis_title="Week",
    yaxis_title="Requests",
    height=350,
    template="plotly_dark"
)

fig.show()

# Run ADF
adf_result = adfuller(series.dropna(), autolag="AIC")

adf_stat = adf_result[0]
p_value  = adf_result[1]

print(f"ADF Statistic: {adf_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# Interpretation
if p_value < 0.05:
    print("Conclusion: Reject H0 → Series is likely stationary (d=0 may be sufficient).")
else:
    print("Conclusion: Fail to reject H0 → Series is likely non-stationary (differencing required).")

In [ ]:
# Time series forecasting requires chronological validation.
# We split the data preserving temporal order (no random shuffle).

split_ratio = 0.8
split_index = int(len(df_weekly) * split_ratio)

train = df_weekly.iloc[:split_index]
test  = df_weekly.iloc[split_index:]

# Plot the split
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=train["week"],
    y=train["n_requests"],
    mode="lines",
    name="Train"
))

fig.add_trace(go.Scatter(
    x=test["week"],
    y=test["n_requests"],
    mode="lines",
    name="Test"
))

fig.update_layout(
    title="Temporal Train/Test Split",
    xaxis_title="Week",
    yaxis_title="Requests",
    height=350,
    template="plotly_dark"
)

fig.show()

print(f"Train size: {len(train)} observations")
print(f"Test size: {len(test)} observations")

# Conclusion:
# The split preserves chronological order, ensuring a realistic forecasting setup.
# The model will be trained on past data and evaluated on unseen future periods.

### **PROPHET Sustentability Validation**

In [ ]:
# PROPHET (Decomposable Time Series Model)

# Core question:
# "Can demand be explained as the sum of structural components?"

# Prophet assumes:

# y(t) = Trend(t) + Seasonality(t) + Noise(t)

# Unlike ARIMA, Prophet does not model autocorrelation structure directly.
# Instead, it models interpretable components of the signal.

# It does NOT require strict stationarity.
# It assumes trend and seasonality can be separated explicitly.

In [ ]:
# Assumptions:

#   - The time series can be decomposed into additive components.
#   - Seasonality is smooth and relatively stable over time.
#   - Trend changes can be approximated by piecewise linear segments.

# Prophet is particularly suited for:
#   - Business demand with strong seasonal structure
#   - Interpretable trend + seasonality separation
#   - Medium-term operational forecasting

In [ ]:
# Rolling long-term mean to approximate trend behavior

long_trend = series.rolling(26).mean()  # ~6 months smoothing

fig_prophet_trend = go.Figure()

fig_prophet_trend.add_trace(go.Scatter(
    x=df_weekly["week"],
    y=series,
    mode="lines",
    name="Original",
    opacity=0.4
))

fig_prophet_trend.add_trace(go.Scatter(
    x=df_weekly["week"],
    y=long_trend,
    mode="lines",
    name="Smoothed Trend (26w)"
))

fig_prophet_trend.update_layout(
    title="Structural Trend Approximation (Prophet Context)",
    xaxis_title="Week",
    yaxis_title="Requests",
    height=350,
    template="plotly_dark"
)

fig_prophet_trend.show()

In [ ]:
# Compare current values to same week last year

seasonal_lag = 52

y_current = series[seasonal_lag:]
y_last_year = series.shift(seasonal_lag)[seasonal_lag:]

fig_prophet_season = go.Figure()

fig_prophet_season.add_trace(go.Scatter(
    x=y_last_year,
    y=y_current,
    mode="markers",
    name="y(t) vs y(t-52)"
))

fig_prophet_season.update_layout(
    title="Yearly Seasonal Consistency (Prophet Context)",
    xaxis_title="Demand t-52",
    yaxis_title="Demand t",
    height=350,
    template="plotly_dark"
)

fig_prophet_season.show()

### **LIGHTGBM Sustentability Validation**

In [ ]:
# LIGHTGBM (Gradient Boosting Regressor for Time Series)

# Core question:
# "Can demand be predicted from engineered temporal features?"

# Unlike ARIMA (stochastic structure) or Prophet (structural decomposition),
# LightGBM treats forecasting as a supervised regression problem.

# It does NOT model time directly.
# It learns relationships between features and target.

In [ ]:
# Assumptions:

#   - No requirement of stationarity.
#   - No assumption of linearity.
#   - Performance depends on feature engineering quality.

# LightGBM learns nonlinear interactions between:

#   - Lagged values
#   - Rolling statistics
#   - Calendar variables
#   - Potential external drivers

# It is flexible but fully dependent on feature construction.

In [ ]:
# Visual inspection of short-term lag dependency

lag_1 = series.shift(1)

fig_lgb_lag = go.Figure()

fig_lgb_lag.add_trace(go.Scatter(
    x=lag_1[1:],
    y=series[1:],
    mode="markers",
    name="y(t) vs y(t-1)"
))

fig_lgb_lag.update_layout(
    title="Short-Term Lag Relationship (LightGBM Context)",
    xaxis_title="Demand t-1",
    yaxis_title="Demand t",
    height=350,
    template="plotly_dark"
)

fig_lgb_lag.show()

In [ ]:
rolling_mean_12 = series.rolling(12).mean()

fig_lgb_roll = go.Figure()

fig_lgb_roll.add_trace(go.Scatter(
    x=df_weekly["week"],
    y=series,
    mode="lines",
    name="Original",
    opacity=0.4
))

fig_lgb_roll.add_trace(go.Scatter(
    x=df_weekly["week"],
    y=rolling_mean_12,
    mode="lines",
    name="Rolling Mean (12w)"
))

fig_lgb_roll.update_layout(
    title="Rolling Feature Relevance (LightGBM Context)",
    xaxis_title="Week",
    yaxis_title="Requests",
    height=350,
    template="plotly_dark"
)

fig_lgb_roll.show()

## Hierarchical Structure Definition

## Temporal Aggregation and Target Definition

## Feature Engineering for Time Series

## Temporal Validation Strategy

## Baseline: Seasonal Naive

## Model 1: ARIMA / SARIMA

## Model 2: Prophet

## Model 3: LightGBM Regressor

## Hierarchical Reconciliation Strategy

## Model Evaluation and Comparison

## Forecast Visualization and Insights

## Business Impact Simulation

## Limitations and Assumptions

## Production Deployment Plan

## Next Steps